In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from keras.utils import to_categorical

from aircraft_dataset import aircraft_dataset_split
from aircraft_reports import display_random_subset
from aircraft_reports import display_class_subset

import sys
sys.path.append('../')
import util.reports as rp

In [ ]:
# Loading dataset
dataset = pd.read_csv('exports/2019-07-28 13-06-28/segment_1.csv')
predictors = dataset.iloc[:,:680].values
responses = dataset.iloc[:,680].values
measurements = dataset.iloc[:,681].values
np.unique(responses)

In [ ]:
# Splits into training and holdout sets
X,y,X_holdout,y_holdout = aircraft_dataset_split(predictors,responses,measurements)

In [ ]:
lc = LabelEncoder()
lc.fit(y)
y = lc.transform(y)
y_holdout = lc.transform(y_holdout)
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)
X_holdout = sc.transform(X_holdout)

In [ ]:
# Display random observations
display_random_subset(X,y,labels=lc.classes_)

In [ ]:
# Display random observations from a particular class
display_class_subset(X,y,category=1,labels=lc.classes_)

In [ ]:
alpha = 0.01
mdl = MLPClassifier(hidden_layer_sizes=(100,), alpha=alpha)
mdl.fit(X,y)

In [ ]:
y_pred = mdl.predict(X)
rp.report_classification(y,y_pred,avg='macro',title='Training')

In [ ]:
y_pred = mdl.predict(X_holdout)
rp.report_classification(y_holdout,y_pred,avg='macro',title='Holdout')

In [ ]:
# Build and trains CNN
alpha = 0.000
mdl = Sequential()
mdl.add(Conv2D(32, kernel_size=2, activation='relu', input_shape=(20,34,1),kernel_regularizer=regularizers.l2(alpha)))
mdl.add(MaxPooling2D(pool_size=2))
mdl.add(Conv2D(64, kernel_size=2, activation='relu',kernel_regularizer=regularizers.l2(alpha)))
mdl.add(MaxPooling2D(pool_size=2))
mdl.add(Flatten())
mdl.add(Dense(100, activation='relu',kernel_regularizer=regularizers.l2(alpha)))
mdl.add(Dropout(rate = 0.2))
mdl.add(Dense(len(np.unique(y)), activation='softmax'))
mdl.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
# Train CNN
mdl.fit(X.reshape((X.shape[0],20,34,1)),y,
        batch_size=200,epochs=100,verbose=1,
        validation_data=(X_holdout.reshape((X_holdout.shape[0],20,34,1)), y_holdout))

In [ ]:
y_pred = mdl.predict(X.reshape((X.shape[0],20,34,1))).argmax(axis=1)
rp.report_classification(y,y_pred,avg='macro',title='Training')

In [ ]:
y_pred = mdl.predict(X_holdout.reshape((X_holdout.shape[0],20,34,1))).argmax(axis=1)
rp.report_classification(y_holdout,y_pred,avg='macro',title='Holdout')